# ProcessCcd 

<br>Owner: **Alex Drlica-Wagner** ([@kadrlica](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@kadrlica))
<br>Last Verified to Run: **2018-08-10**
<br>Verified Stack Release: **v16.0**

## Learning Objectives:

This notebook seeks to teach users how to unpack a command line task, specifically `processCcd.py`, and access the python API functionality that is being called. This notebook is a digression from Justin Myles script that runs the HSC rerun processing [link].

After working through this tutorial you should be able to:

* Find the source code for a command line task
* Investigate and run those tasks in python

## Logistics
This notebook is intended to be runnable on `lsst-lspdev.ncsa.illinois.edu` from a local git clone of https://github.com/LSSTScienceCollaborations/StackClub.

In [1]:
import os
import pydoc

## Command Line Processing

In this first step we perform command line processing of images following the getting started tutorials [here](https://pipelines.lsst.io/getting-started/data-setup.html#) and [here](https://pipelines.lsst.io/getting-started/processccd.html). We are specifically interested in digging into the following line:

```
processCcd.py $DATADIR --rerun processCcdOutputs --id
```

We start by tracking down the location of the `processCcd.py` shell script

In [2]:
!(which processCcd.py)

/opt/lsst/software/stack/stack/miniconda3-4.3.21-10a4fa6/Linux64/pipe_tasks/16.0+1/bin/processCcd.py


This is the proverbial "end of the thread". Our goal is to pull on this thread to unravel the python/C++ functions that are being called under the hood. We start by taking a peak in this script

In [3]:
!cat $(which processCcd.py)

#!/usr/bin/env python
#
# LSST Data Management System
# Copyright 2008, 2009, 2010 LSST Corporation.
#
# This product includes software developed by the
# LSST Project (http://www.lsst.org/).
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.    See the
# GNU General Public License for more details.
#
# You should have received a copy of the LSST License Statement and
# the GNU General Public License along with this program.  If not,
# see <http://www.lsstcorp.org/LegalNotices/>.
#
from lsst.pipe.tasks.processCcd import ProcessCcdTask

ProcessCcdTask.parseAndRun()


Ok, this hasn't gotten us very far, but after getting through the stock header, we now have the next link in our chain:
```
from lsst.pipe.tasks.processCcd import ProcessCcdTask
```

There are two ways we can proceed from here. One is to [Google](http://lmgtfy.com/?q=lsst.pipe.tasks.processCcd) `lsst.pipe.tasks.processCcd`, which will take us to this [doxygen page](http://doxygen.lsst.codes/stack/doxygen/x_masterDoxyDoc/classlsst_1_1pipe_1_1tasks_1_1process_ccd_1_1_process_ccd_task.html) and/or the soure code on [GitHub](https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/processCcd.py). 

The second approach is to do the import the class oursleves and try to investigate it interactively.


In [4]:
import lsst.pipe.tasks.processCcd
from lsst.pipe.tasks.processCcd import ProcessCcdTask, ProcessCcdConfig

We can get to the source code for these classes directly using the [`stackclub` toolkit module](https://stackclub.readthedocs.io/), as shown in the [FindingDocs.ipynb](https://github.com/LSSTScienceCollaborations/StackClub/blob/master/GettingStarted/FindingDocs.ipynb)

In [12]:
from stackclub import where_is
where_is(ProcessCcdConfig)

[lsst.pipe.tasks.processCcd](https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/processCcd.py)

[lsst.pipe.tasks.processCcd](https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/processCcd.py)


Next we can create an instance of the `ProcessCcdConfig` and try calling the `help` method (commented out for brevity). What we are really interested in are the "Data descriptors", which we can print directly after capturing the documentation output by `help`.

In [48]:
config = ProcessCcdConfig()
#help(config)
helplist = pydoc.render_doc(config).split('\n')
print('\n'.join(helplist[18:47]))

 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  isr
 |      Task to perform instrumental signature removal or load a post-ISR image; ISR consists of:
 |      - assemble raw amplifier images into an exposure with image, variance and mask planes
 |      - perform bias subtraction, flat fielding, etc.
 |      - mask known bad pixels
 |      - provide a preliminary WCS
 |       (`ConfigurableInstance`, default ``<class 'lsst.ip.isr.isrTask.IsrTaskConfig'>``)
 |  
 |  charImage
 |      Task to characterize a science exposure:
 |      - detect sources, usually at high S/N
 |      - estimate the background, which is subtracted from the image and returned as field "background"
 |      - estimate a PSF model, which is added to the exposure
 |      - interpolate over defects and cosmic rays, updating the image, variance and mask planes
 |       (`ConfigurableInstance`, default ``<class 'lsst.pipe.tasks.characterizeImage.Chara

The first step is to try to get at the documentation through the `help` function (commented out below for brevity). We can find 

The `ProcessCcdConfig` object contains of both raw configurables like `doCalibrate` and other configs, like `isr`. To investigate one of these in more detail, we can import it and query it's "Data descriptors".


In [70]:
from lsst.ip.isr.isrTask import IsrTask, IsrTaskConfig
print('\n'.join(pydoc.render_doc(IsrTaskConfig).split('\n')[16:40]) + '...')

 |  
 |  Data descriptors defined here:
 |  
 |  doBias
 |      Apply bias frame correction? (`bool`, default ``True``)
 |  
 |  doDark
 |      Apply dark frame correction? (`bool`, default ``True``)
 |  
 |  doFlat
 |      Apply flat field correction? (`bool`, default ``True``)
 |  
 |  doFringe
 |      Apply fringe correction? (`bool`, default ``True``)
 |  
 |  doDefect
 |      Apply correction for CCD defects, e.g. hot pixels? (`bool`, default ``True``)
 |  
 |  doAddDistortionModel
 |      Apply a distortion model based on camera geometry to the WCS? (`bool`, default ``True``)
 |  
 |  doWrite
 |      Persist postISRCCD? (`bool`, default ``True``)
 |  ...


These configurationas are pretty self-explanitory, but say that we really want to understand what `doFringe` is doing. Inorder to get that information we need to go to the source code.

In [63]:
where_is(IsrTaskConfig)

[lsst.ip.isr.isrTask](https://github.com/lsst/ip_isr/blob/master/python/lsst/ip/isr/isrTask.py)

[lsst.ip.isr.isrTask](https://github.com/lsst/ip_isr/blob/master/python/lsst/ip/isr/isrTask.py)


We can then search this file for `doFringe` and we find [several lines](https://github.com/lsst/ip_isr/blob/cc4efb7d763d3663c9e989339505df9654f23fd9/python/lsst/ip/isr/isrTask.py#L597-L598) that look like this:

        if self.config.doFringe and not self.config.fringeAfterFlat:
            self.fringe.run(ccdExposure, **fringes.getDict())
            
If we want to go deeper to see what `fringe.run` does, we can repeat the above process

In [75]:
isr_task = IsrTask()
print(isr_task.fringe)
import  lsst.ip.isr.fringe
where_is(lsst.ip.isr.fringe.FringeTask)

[lsst.ip.isr.fringe](https://github.com/lsst/ip_isr/blob/master/python/lsst/ip/isr/fringe.py)

[lsst.ip.isr.fringe](https://github.com/lsst/ip_isr/blob/master/python/lsst/ip/isr/fringe.py)


We finally make our way to the source code for [FringeTask.run](https://github.com/lsst/ip_isr/blob/cc4efb7d763d3663c9e989339505df9654f23fd9/python/lsst/ip/isr/fringe.py#L104), which gives us details on the fringe subtraction.